In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Apr  8 13:05:28 2024

@author: salman
"""
import csv
import ast

import requests
import json
import openai
import os
import re
import random
import time

In [2]:
"""
All Haoting's code is in this block  !

Set up guide

Step 1: install llama_cpp

# CPU
pip install --upgrade --quiet  llama-cpp-python
# Mac Metal
CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python
# Cuda
CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

Reference
<https://python.langchain.com/docs/integrations/llms/llamacpp/>
<https://llama-cpp-python.readthedocs.io/en/latest/api-reference/>

Step 2: donwload the model

<https://huggingface.co/TheBloke/Llama-2-7B-GGUF/tree/main>

"""

# Libraries
from llama_cpp import Llama
import pandas as pd
from typing import Union, Tuple

# Model initialisation parameters
MODEL_PATH = "../llms/Meta-Llama-3-8B.Q4_K_M.gguf"
N_GPU_LAYERS = -1 # GPU acceleration
SEED = 2024
N_CONTEXT = 4096

# Model prompting parameters
TEMPERATURE = 0.1 # creativity of output
MAX_TOKENS = 100 # maximum token generated by the llm
TOP_P = 0.95 # sampling the top p probability mass
STOP = [] # a list of output token that stop the llm to generate more response

# Prompt construction parameters
MAX_CANDIDATES_PER_PROMPT = 30 # if there are too many candidate, split into several prompt

# Global variables
llm = None

# Set random function
random.seed(SEED)


def load_llm():
    """ Initialise llama2 """
    global llm
    llm = Llama(model_path=MODEL_PATH, 
                n_gpu_layers=N_GPU_LAYERS,
                seed=SEED,
                n_ctx = N_CONTEXT)


def llama2_query_best_candid(entity: str, candidates: pd.DataFrame | list, context: str, verbose) -> str | None:
    """
    Ask the llm to choose a Wiki ID from a number of candidates

    if verbose = Ture, display candidates
    """

    # Convert to data frame
    if type(candidates) == list:
        candidates = pd.DataFrame(candidates, columns=['WikiID', 'instanceOrSubclassOf', 'hasDescription'])
        candidates.set_index('WikiID', inplace=True)
        candidates['hasLabel'] = entity
        
        # Drop r# Drop rows with None or empty list/set in 'hasDescription'
        candidates = candidates.dropna(subset=['hasDescription'])  # Drop None values
        candidates = candidates[candidates['hasDescription'].apply(lambda x: x != [] and x != set())]
        print("The following table shows all candidates retrieved by SPARQL query.")
        display(candidates)


    # For small number of candidates, do it in a single prompt
    if len(candidates) <= MAX_CANDIDATES_PER_PROMPT:
        return llama2_query_best_candid_small(entity, context, candidates, verbose)
    
    # For the number of candiates larger than the threshold, split the candidate set
    if verbose:
        print(f"\n[!!!] There are too many candidates (> {MAX_CANDIDATES_PER_PROMPT}) for the llm to choose. The candidates set will be split and multiple prompts will be made.\n")

    candidates_partition = [candidates[i:i + MAX_CANDIDATES_PER_PROMPT] for i in range(0, candidates.shape[0], MAX_CANDIDATES_PER_PROMPT)]
   
    best_candids = []
    for candidate_subset in candidates_partition:
        result = llama2_query_best_candid_small(entity, context, candidate_subset, verbose)
        if result is not None:
            best_candids.append(result)
    
    # Gather the correct answer from each subset of candidates
    if len(best_candids) == 0:
        return None
    elif len(best_candids) == 1:
        return best_candids[0]
    elif len(best_candids) > 1:
        new_candidates = candidates.loc[best_candids]
        return llama2_query_best_candid(entity, new_candidates, context, verbose=verbose)


def llama2_query_best_candid_small(entity: str, context: str, candidates: pd.DataFrame, verbose: bool) -> str | None:
    """
    Ask the llm to choose a Wiki ID from no more than MAX_CANDIDATES_PER_PROMPT candidates

    verbose = True, then print the prompt, response, and parsed output
    """
    global llm

    # No candidate, return None
    if len(candidates) == 0:
        return None
    
    # One candidate, return the only one
    if len(candidates) == 1:
        return candidates.iloc[0].name
    
    # More than one, ask llm
    assert(len(candidates) <= MAX_CANDIDATES_PER_PROMPT)

    # Construct candidate string
    candidates_string = ''
    count = 1
    for index, row in candidates.iterrows(): 
        candidates_string = candidates_string + f"Option {str(count)}: {row['hasDescription'][0]}\n"
        count += 1

    # Construct prompt
    prompt = f"""As a linguistic specialist, consider the following task:
Given the sentence "{context}", what does the entity "{entity}" refer to in this sentence? Choose the most relevant and specific answer from the following options. Explain your answer:

{candidates_string}
The correct answer is:

Option """
    
    # Check the tokens is below the context windows
    tokens = llm.tokenizer().tokenize(bytes(prompt, 'utf-8'))
    if (len(tokens) + MAX_TOKENS) > N_CONTEXT:
        return None
    
    # Get response
    response = llm(
      prompt,
      max_tokens=MAX_TOKENS,
      temperature=TEMPERATURE,
      top_p=TOP_P,
      stop=STOP,
      echo=False # Echo the prompt back in the output
      )
    
    response = response['choices'][0]['text']

    if verbose:
        print()
        print(prompt + response)
        print()
    
    # Parse response
    match = re.search(r'(\d+)', response)

    # Prevent the llm to generate a non-existing ID
    if match:
        if int(match.group(1)) - 1 < len(candidates):
            return candidates.index[int(match.group(1)) - 1]
    return None

# Execute Code
load_llm()

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ../llms/Meta-Llama-3-8B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   7:                 llama.rope.dimension_count u

In [3]:
from collections import defaultdict

# Definintion of  endpoint and headers
endpoint_url = "https://query.wikidata.org/sparql"
headers = {"Accept": "application/sparql-results+json"}

def sparql_query_type(label):
    '''
    Input: Entity Label
    Functionality: Extracting the URI with relevant information from Wikidata SPARQL endpoint
    Return: List of QIDs for entity_label.
    '''
    
    
    query_des = 'SELECT ?entity ?type ?typeLabel ?subclassLabel ?entityDescription WHERE {?entity rdfs:label \"'+label+'\"@en . OPTIONAL { ?entity wdt:P31 ?type .} OPTIONAL {?entity wdt:P279 ?subclass.} FILTER (!bound(?type) || ?type != wd:Q4167410) SERVICE wikibase:label { bd:serviceParam wikibase:language "en".} }'
    

    response = requests.get(endpoint_url, headers=headers, params={'query': query_des, 'format': 'json'})
    items_with_classes = defaultdict(list)
    items_with_descr = defaultdict(list)
    # Process the response
    if response.ok:
        data = response.json()
        for result in data['results']['bindings']:
            #print(result)
        
        
            entity_uri = result['entity']['value'].split('/')[-1]
            type_label = result.get('typeLabel')
            subclass = result.get('subclassLabel')
            desc = result.get('entityDescription')
        
            type_uri = ''
            class_uri = ''
            descr = ''
        
            if type_label:
                type_uri = result['typeLabel']['value']
        
            if subclass:
                class_uri = result['subclassLabel']['value']
            
            if desc:
                descr = result['entityDescription']['value']
        
        
            if type_uri !='':
                items_with_classes[entity_uri].append(type_uri)
            
            if class_uri != '':
                items_with_classes[entity_uri].append(class_uri)
        
            if descr != '':
                if entity_uri not in items_with_descr or not items_with_descr[entity_uri]:
                    items_with_descr[entity_uri].append(descr)
    
    
    return [(entity_uri, classes, items_with_descr[entity_uri]) for entity_uri, classes in items_with_classes.items()]   



def find_case_sensitive_term(text, search_term):
    # Compile a regex pattern for case-insensitive matching
    pattern = re.compile(re.escape(search_term), re.IGNORECASE)
    
    # Find all matches in the text
    matches = pattern.findall(text)
    
    # Return the list of matches
    return matches[0]

In [4]:
# haoting's modified code block

def run(entity_label: str, query_text: str, verbose=True) -> Tuple[Union[str, None], bool, float, float, list]:
    """Given a label and its context, return an Wikidata ID or None"""

    sparql_recalled = False

    # Step 1: make the label case-sensitive
    if entity_label in query_text:
        entity_label = entity_label
    else:
        entity_label = find_case_sensitive_term(query_text, entity_label)

    # Step 2: Try to get candidates. If it fails, retry up to 5 times.
    start_time = time.time()
    trials = 1
    candidates = sparql_query_type(entity_label)
    while not candidates and trials < 5:
        candidates = sparql_query_type(entity_label)
        trials += 1
    if candidates:
        sparql_recalled = True
    sparql_time = time.time() - start_time
    
    # Step 3: Ask the llm to choose a Wiki ID from the candidates
    start_time = time.time()
    ID = llama2_query_best_candid(entity_label, candidates, query_text, verbose=True)
    llm_time = time.time() - start_time

    # Step 4: if ID none, search for the lower-cased or capitalised variation of the entity.
    if ID is None:
        if entity_label == entity_label.lower():
            entity_label = entity_label.capitalize()
        else:
            entity_label = entity_label.lower()
        
        # Step 4.1: Try to get candidates. If it fails, retry up to 3 times.
        start_time = time.time()
        candidates = sparql_query_type(entity_label)
        trials = 1
        while not candidates and trials < 3:
            candidates = sparql_query_type(entity_label)
            trials += 1
        if candidates:
            sparql_recalled = True
        sparql_time += time.time() - start_time
        
        # Step 4.2: ask the llm for a correct Wiki ID
        start_time = time.time()
        ID = llama2_query_best_candid(entity_label, candidates, query_text, verbose=True)
        llm_time += time.time() - start_time
    

    # Print epilogue
    if verbose:
        print('\n' + ("=" * 100))
        print("NEL task ends\n")
        print(f"Predicted ID = '{ID}'")
        print(f"Actual ID = '{c_id}'")
        if ID == c_id:
            print("Wiki ID Matched")
        elif ID == None:
            print("Wiki ID Not Found")
        else:
            print("Wiki ID Not Matched")
        print(("=" * 100) + '\n')

    return ID, sparql_recalled, sparql_time, llm_time, candidates



########################################################
##              Experiment Start Here                 ##
########################################################

# Initialise variables
file_path = './test_full.json' # path to the dataset
n = 0 # number of sample tested
P, R, A, F = 0, 0, 0, 0 # precision, recall, accuracy, f-score
R_SPARQL = 0 # recall for SPARQL query

with open(file_path, 'r') as file:
    data = json.load(file)
    data_samples = random.sample(data, 100)

    # Loop through all sample
    for content in data_samples:
        
        # Increment the number of sample tested
        n += 1
        recalled = False
        entity_label = content['string']
        query_text = content['text']
        c_id = content['correct_id']
        
        # Print prologue
        print('\n' + ("=" * 100))
        print(f"({n}/{len(data_samples)}) NEL task starts\n")
        print(f"Entity = '{entity_label}'")
        print(f"Context = '{query_text}'")
        print(("=" * 100) + '\n')
        
        ID, sparql_recalled, sparql_time, llm_time, candidates = run(entity_label, query_text)

        # Compute precision
        if ID is not None and c_id == ID:
            P += 1
        
        # Compute recall (llm)
        if ID is not None:
            R += 1

        # Compute accuracy
        if c_id == ID:
            A += 1
        
        # Compute recall (sparql)
        if sparql_recalled:
            R_SPARQL += 1
                
        summary = n, query_text, candidates, entity_label, c_id, ID, sparql_time, llm_time
        
        with open('./Sample1.csv', 'a') as file:
              writer = csv.writer(file, delimiter=',')
              writer.writerow(summary)

# Compute Precision
P = P / R * 100

# Compute Recall
R = R / n * 100

# Compute Accuracy
A = A / n * 100

# Compute SPARQL Recall
R_SPARQL = R_SPARQL / n * 100

print("\n###########  PERFORMANCE   ###########")    

print(f"Precision: {P} %")
print(f"Recall: {R} %")
print(f"Accuracy: {A} %")
print(f"SPARQL Recall: {R_SPARQL} %")
print(f"F-Score: {2 * P * R / (P + R)} %")

print("#####################################")   



(1/100) NEL task starts

Entity = 'east asia'
Context = ' invasion, the conquest of East Asia by Japan that cuts the Chinese conquests in Russia and Europe down to the limits of the former Austro-Hungarian empire, and the Japanese '

The following table shows all candidates retrieved by SPARQL query.


,instanceOrSubclassOf,hasDescription,hasLabel
WikiID,,,
Q27231,[geographic region],[eastern region of Asia],East Asia
Q5327728,[album],[album by Miyuki Nakajima],East Asia
Q15761181,[scientific journal],[journal],East Asia
Q58558114,[scholarly article],[article],East Asia
Q86075977,[scholarly article],[scientific article published on 01 March 2014],East Asia
Q125592707,[scholarly article],[scientific article published in 2009],East Asia
Q125857107,[scholarly article],[scientific article published in 2010],East Asia



llama_print_timings:        load time =    5477.40 ms
llama_print_timings:      sample time =      19.72 ms /    70 runs   (    0.28 ms per token,  3550.42 tokens per second)
llama_print_timings: prompt eval time =    5476.63 ms /   164 tokens (   33.39 ms per token,    29.95 tokens per second)
llama_print_timings:        eval time =    1574.61 ms /    69 runs   (   22.82 ms per token,    43.82 tokens per second)
llama_print_timings:       total time =    7437.24 ms /   233 tokens



As a linguistic specialist, consider the following task:
Given the sentence " invasion, the conquest of East Asia by Japan that cuts the Chinese conquests in Russia and Europe down to the limits of the former Austro-Hungarian empire, and the Japanese ", what does the entity "East Asia" refer to in this sentence? Choose the most relevant and specific answer from the following options. Explain your answer:

Option 1: eastern region of Asia
Option 2: album by Miyuki Nakajima
Option 3: journal
Option 4: article
Option 5: scientific article published on 01 March 2014
Option 6: scientific article published in 2009
Option 7: scientific article published in 2010

The correct answer is:

Option 1: eastern region of Asia
Explanation: The entity "East Asia" refers to the eastern region of Asia. This can be inferred from the context of the sentence, which mentions Japan and China as two countries located in this region.
Note that the other options are not relevant or specific enough to refer to t

,instanceOrSubclassOf,hasDescription,hasLabel
WikiID,,,
Q481410,[film],[2011 film by Tim Fehlbaum],Hell
Q624974,[film],[2010 film by Luis Estrada],Hell
Q1213581,[film],[1994 film by Claude Chabrol],Hell
Q1516314,[impact crater],[lunar crater],Hell
Q1525785,[album],[2008 studio album by Venom],Hell
Q1602602,[musical group],[British heavy metal band],Hell
Q1602605,[unincorporated community in the United States],[unincorporated community in Livingston County...,Hell
Q1602613,[film],[2005 film by Tanit Jitnukul],Hell
Q1602619,"[village, urban area in Norway]","[settlement in Stjørdal Municipality, Norway]",Hell


...

Llama.generate: prefix-match hit

llama_print_timings:        load time =    5477.40 ms
llama_print_timings:      sample time =      28.06 ms /   100 runs   (    0.28 ms per token,  3563.79 tokens per second)
llama_print_timings: prompt eval time =     373.60 ms /   139 tokens (    2.69 ms per token,   372.06 tokens per second)
llama_print_timings:        eval time =    2244.06 ms /    99 runs   (   22.67 ms per token,    44.12 tokens per second)
llama_print_timings:       total time =    3179.31 ms /   238 tokens



As a linguistic specialist, consider the following task:
Given the sentence " programming supporting the Cultural Revolution under Mao Zedong. At the time, it had a relay station in Albania, but nowhere else. Radio Peking, by that name, remained on ", what does the entity "Mao Zedong" refer to in this sentence? Choose the most relevant and specific answer from the following options. Explain your answer:

Option 1: 1st Chairman of the Chinese Communist Party and founder of the People's Republic of China (1893–1976)
Option 2: Chinese television series
Option 3: encyclopedia article
Option 4: 1972, print, screen printing

The correct answer is:

Option 1: 1st Chairman of the Chinese Communist Party and founder of the People's Republic of China (1893–1976)
Explanation: The entity "Mao Zedong" refers to the first chairman of the Chinese communist party and founder of the people's republic of china. This can be inferred from the context of the sentence, which mentions his involvement in the

,instanceOrSubclassOf,hasDescription,hasLabel
WikiID,,,
Q395,"[academic major, natural science, academic dis...",[field of study],mathematics
Q2798931,[school subject],[school subject in the Netherlands],mathematics
Q71257062,[genre],[book genre],mathematics


Llama.generate: prefix-match hit

llama_print_timings:        load time =    5477.40 ms
llama_print_timings:      sample time =      27.40 ms /    94 runs   (    0.29 ms per token,  3430.53 tokens per second)
llama_print_timings: prompt eval time =     232.97 ms /    69 tokens (    3.38 ms per token,   296.17 tokens per second)
llama_print_timings:        eval time =    2088.34 ms /    93 runs   (   22.46 ms per token,    44.53 tokens per second)
llama_print_timings:       total time =    2856.75 ms /   162 tokens



As a linguistic specialist, consider the following task:
Given the sentence "In mathematics, ", what does the entity "mathematics" refer to in this sentence? Choose the most relevant and specific answer from the following options. Explain your answer:

Option 1: field of study
Option 2: school subject in the Netherlands
Option 3: book genre

The correct answer is:

Option 1: field of study
Explanation: The entity "mathematics" refers to a field of study, as it is used in the sentence. This can be inferred from the context of the sentence and the fact that mathematics is a subject studied at universities.
Note: In this task, we are not concerned with the specific meaning of the word "mathematics", but rather with its referent in the given sentence. The correct answer should reflect this focus on reference.



NEL task ends

Predicted ID = 'Q395'
Actual ID = 'Q395'
Wiki ID Matched


###########  PERFORMANCE   ###########
Precision: 85.55555555555556 %
Recall: 90.0 %
Accuracy: 77.0 %
SPA